In [42]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import date, timedelta
from pandas.tseries.offsets import *
import calendar

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()

data_path = "../excel/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
xsl_path = "\\Users\\PC1\\OneDrive\\A5\\Excel\\"

year = 2025
quarter = 3

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2025, 11, 30), datetime.date(2025, 11, 29))

In [3]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
yesterday = yesterday.date()
print(f'yesterday: {yesterday}')

today: 2025-11-30
yesterday: 2025-11-28


In [5]:
format_dict = {    
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}','qtrly':'{:.4f}',
    'price':'{:.2f}',    
    'amount':'{:,.2f}','net':'{:,.2f}','cost_amt':'{:,.2f}',
    'yield':'{:,.2f}%','pct':'{:,.2f}%',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
              }

### New dividend stock

In [4]:
name = 'LHK'
sql = """
SELECT * 
FROM DIVIDEND
WHERE name = '%s'
"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual


In [5]:
sqlIns = """
INSERT INTO dividend
VALUES('LHK',0,0.20,0,0.45,0.65,0,0,00,'2022-11-25','2022-12-09','',1)
"""
rp = const.execute(sqlIns)
rp.rowcount

1

### Start of Update dividend

In [7]:
name = 'AIMIRT'
sql = """
SELECT * FROM DIVIDEND WHERE name = '%s'"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,AIMIRT,0.2150,0.2150,0.2050,0.2150,0.8500,"17,500",2025-08-19,2025-09-05,,0


In [9]:
def update_dividend1(q3, XDATE, PAIDDATE, actual, name):
    sql = text("UPDATE dividend SET q3 = :q3, dividend = q1+q2+q3+q4, XDATE = :xdate, PAIDDATE = :pdate, actual = :actual WHERE name = :name")
    rp = const.execute(sql, {"q3": q3, "xdate": XDATE, "pdate": PAIDDATE, "actual": actual, "name": name})
    return "Records updated = " + str(rp.rowcount)

In [11]:
q3 = 0.2050
XDATE = '2025-11-19'
PAIDDATE = '2025-12-04'
actual = 1

update_dividend1(q3,XDATE,PAIDDATE,actual,name)

'Records updated = 1'

In [13]:
sql = """
SELECT * FROM DIVIDEND WHERE name = '%s'"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,AIMIRT,0.2150,0.2050,0.2050,0.2150,0.8400,"17,500",2025-11-19,2025-12-04,,1


In [26]:
def update_dividend3(shares,q3,actual,name):
    sql = "UPDATE dividend SET shares = %s, q3 = %s, dividend=q1+q2+q3+q4, actual = %s WHERE name = '%s'"
    sql = sql % (shares, q3, actual, name)
    rp = const.execute(sql)
    return "Records updated = " + str(rp.rowcount)

In [27]:
name = '3BBIF'
sql = """
SELECT * 
FROM DIVIDEND
WHERE name = '%s'
"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,3BBIF,0.1600,0.0000,0.0000,0.0000,0.1600,"120,000",2024-08-16,2024-09-23,,1


In [12]:
shares = 120_000
q3 = 0
actual = 1

In [13]:
update_dividend3(shares,q3,actual,name)

'Records updated = 1'

### Toggle actual status

In [14]:
name = 'TFFIF'
actual = 0
sqlUpd = "UPDATE dividend SET actual = %s WHERE name = '%s'"
sqlUpd = sqlUpd % (actual, name)
rp = const.execute(sqlUpd)
rp.rowcount

1

### Delete dividend record

In [15]:
sqlDel = '''
DELETE FROM dividend
WHERE name IN ("DOHOME")
'''
rp = const.execute(sqlDel)
rp.rowcount

0

### Start of output to cloud

In [15]:
sql = """
SELECT Y.NAME AS name,  Q3 AS qtrly, SHARES, XDATE, PAIDDATE, 
P.price AS price, Y.DIVIDEND, ACTUAL, B.price * B.volbuy AS cost_amt
FROM dividend AS Y, price AS P, buy AS B
WHERE Y.name = P.name 
AND Y.name = B.name 
AND P.date = '%s'
AND (Q3 > 0 OR Y.name IN ('GVREIT','TFFIF')) 
ORDER BY name
"""
sql = sql % yesterday 
print(sql)


SELECT Y.NAME AS name,  Q3 AS qtrly, SHARES, XDATE, PAIDDATE, 
P.price AS price, Y.DIVIDEND, ACTUAL, B.price * B.volbuy AS cost_amt
FROM dividend AS Y, price AS P, buy AS B
WHERE Y.name = P.name 
AND Y.name = B.name 
AND P.date = '2025-11-28'
AND (Q3 > 0 OR Y.name IN ('GVREIT','TFFIF')) 
ORDER BY name



In [17]:
cols = 'name qtrly shares amount net xdate paiddate cost_amt pct actual'.split()

In [19]:
dividend = pd.read_sql(sql, const)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend['amount'] = round(dividend['shares'] * dividend['qtrly'], 2)
dividend['net'] = round(dividend['amount'] * 0.9, 2)
dividend['pct'] = round(dividend['net'] / dividend['cost_amt'] * 100, 2)
dividend[cols].style.format(format_dict)

,name,qtrly,shares,amount,net,xdate,paiddate,cost_amt,pct,actual
0,3BBIF,0.1372,"120,000","16,464.00","14,817.60",2025-11-14,2025-12-22,"1,212,000.00",1.22%,1
1,AIMIRT,0.2050,"17,500","3,587.50","3,228.75",2025-11-19,2025-12-04,"185,500.00",1.74%,1
2,CPNREIT,0.0691,"55,000","3,800.50","3,420.45",2025-11-25,2025-12-11,"990,000.00",0.35%,1
3,DIF,0.2222,"45,000","9,999.00","8,999.10",2025-11-17,2025-12-08,"571,500.00",1.57%,1
4,GVREIT,0.1965,"69,000","13,558.50","12,202.65",2025-12-09,2025-12-24,"534,750.00",2.28%,1
5,IVL,0.1750,"7,200","1,260.00","1,134.00",2025-11-25,2025-12-11,"288,000.00",0.39%,1
6,JMART,0.1300,"6,800",884.00,795.60,2025-11-24,2025-12-11,"224,400.00",0.35%,1
7,ORI,0.0210,"50,000","1,050.00",945.00,2025-11-27,2025-12-15,"450,000.00",0.21%,1
8,PTG,0.1000,"3,600",360.00,324.00,2025-03-10,2025-05-16,"41,040.00",0.79%,0
9,RCL,0.5000,"27,000","13,500.00","12,150.00",2025-11-20,2025-12-04,"1,047,060.00",1.16%,1


In [21]:
dividend.net.sum()

69484.04999999999

## Output Excel file

In [30]:
excel_fmt = dividend.sort_values(['name'], ascending=[True]).copy()
excel_fmt['xdate'] = pd.to_datetime(excel_fmt['xdate'])
excel_fmt['paiddate'] = pd.to_datetime(excel_fmt['paiddate'])
excel_fmt['div_amt'] = excel_fmt['qtrly'] * excel_fmt['shares']
excel_fmt['net_amt'] = excel_fmt['div_amt'] * 0.9
excel_fmt['xdate'] = excel_fmt['xdate'].dt.strftime('%Y-%m-%d')
excel_fmt['paiddate'] = excel_fmt['paiddate'].dt.strftime('%Y-%m-%d')

In [32]:
colt = 'name qtrly shares div_amt net_amt xdate paiddate actual'.split()
excel_fmt[colt].sort_values(['actual','xdate','name'],ascending=[False,False,True])

,name,qtrly,shares,div_amt,net_amt,xdate,paiddate,actual
4,GVREIT,0.1965,69000,13558.5,12202.65,2025-12-09,2025-12-24,1
10,TFFIF,0.1128,20000,2256.0,2030.40,2025-12-08,2025-12-25,1
7,ORI,0.0210,50000,1050.0,945.00,2025-11-27,2025-12-15,1
2,CPNREIT,0.0691,55000,3800.5,3420.45,2025-11-25,2025-12-11,1
5,IVL,0.1750,7200,1260.0,1134.00,2025-11-25,2025-12-11,1
6,JMART,0.1300,6800,884.0,795.60,2025-11-24,2025-12-11,1
11,WHAIR,0.1325,50000,6625.0,5962.50,2025-11-24,2025-12-22,1
12,WHART,0.1930,20000,3860.0,3474.00,2025-11-21,2025-12-11,1
9,RCL,0.5000,27000,13500.0,12150.00,2025-11-20,2025-12-04,1
1,AIMIRT,0.2050,17500,3587.5,3228.75,2025-11-19,2025-12-04,1


In [34]:
excel_fmt.net.sum()

69484.04999999999

In [44]:
file_name = '22-div-25q3.xlsx'
xsl_file = os.path.join(xsl_path, file_name)

excel_fmt[colt].sort_values(['actual','xdate','name'],ascending=[False,False,True]).to_excel(xsl_file, index=False)

### End of output to cloud

In [46]:
const.commit()
const.close()